In [8]:
import pandas as pd
import numpy as np

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from cataclop.ml import preprocessing
from cataclop.ml import exploration

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

from cataclop.ml.pipeline import factories

In [11]:
dataset = factories.Dataset.factory('default', params={
    'from': '2020-01-01',
    'to': '2020-02-01',
})
dataset.load()

In [12]:
df = dataset.players
df.describe()

,age,earnings,final_odds,final_odds_offline,final_odds_ref,final_odds_ref_offline,final_odds_ref_unibet,final_odds_unibet,handicap_distance,handicap_weight,herder_id,horse_id,jockey_id,num,owner_id,placed_2_count,placed_3_count,placed_count,placed_dividend,placed_earnings,position,post_position,prev_year_earnings,race_count,race_id,time,trainer_id,victory_count,victory_earnings,winner_dividend,year_earnings,declared_player_count,distance,num_race,num_bis,prize,session_id,hippodrome_id,num_session,hist_1_pos,hist_2_pos,hist_3_pos,hist_4_pos,hist_5_pos,hist_6_pos,speed,win,final_odds_ref_inv,race_count_mean,race_count_std,race_count_amin,race_count_amax,hist_1_pos_mean,hist_1_pos_std,hist_1_pos_amin,hist_1_pos_amax,hist_2_pos_mean,hist_2_pos_std,hist_2_pos_amin,hist_2_pos_amax,hist_3_pos_mean,hist_3_pos_std,hist_3_pos_amin,hist_3_pos_amax,victory_count_mean,victory_count_std,victory_count_amin,victory_count_amax,placed_2_count_mean,placed_2_count_std,placed_2_count_amin,placed_2_count_amax,placed_3_count_mean,placed_3_count_std,placed_3_count_amin,placed_3_count_amax,victory_earnings_mean,victory_earnings_std,victory_earnings_amin,victory_earnings_amax,placed_earnings_mean,placed_earnings_std,placed_earnings_amin,placed_earnings_amax,prev_year_earnings_mean,prev_year_earnings_std,prev_year_earnings_amin,prev_year_earnings_amax,handicap_distance_mean,handicap_distance_std,handicap_distance_amin,handicap_distance_amax,handicap_weight_mean,handicap_weight_std,handicap_weight_amin,handicap_weight_amax,final_odds_ref_mean,final_odds_ref_std,final_odds_ref_amin,final_odds_ref_amax,final_odds_ref_offline_mean,final_odds_ref_offline_std,final_odds_ref_offline_amin,final_odds_ref_offline_amax,race_count_r,hist_1_pos_r,hist_2_pos_r,hist_3_pos_r,victory_count_r,placed_2_count_r,placed_3_count_r,victory_earnings_r,placed_earnings_r,prev_year_earnings_r,handicap_distance_r,handicap_weight_r,final_odds_ref_r,final_odds_ref_offline_r,odds_0,odds_1,odds_2,odds_3,odds_4,odds_5,odds_6,odds_7,odds_8,odds_9,odds_10,odds_11,odds_12,odds_13,odds_14,odds_15,odds_16,odds_17,odds_18,odds_19
count,14139.000000,1.413900e+04,13945.000000,12518.000000,14139.000000,14139.000000,13517.000000,13541.000000,14139.000000,14139.000000,10073.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,11597.000000,14139.000000,14139.000000,14139.000000,14139.000000,5456.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,1503.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.000000,14139.00000,14139.000000,14139.000000,14139.0,14139.0
mean,5.521041,6.969967e+06,35.200620,33.801490,8681.845435,17764.793111,29.224140,32.867602,1332.248108,10.70606